## 09 한글 텍스트 처리 - 네이버 영화 평점 감성 분석

###  한글 NLP 처리의 어려움

In [19]:
# 일반적으로 한글 언어 처리는 영어 등의 라틴어 처리보다 어렵다.
# 그 주된 원인은 '띄어쓰기'와 '다양한 조사' 때문이다.
# 조사는 주어나 목적어를 위해 추가되며, 워낙 경우의 수가 많기 때문에 
# 어근 추출(Stemming/Lemmatization) 등의 전처리 시 제거하기가 까다롭다.
# 이러한 이슈로 인해서 상대적으로 라틴어 계열의 언어보다 NLP 처리가 어려운 문제가 있다.

### 데이터 로딩

In [20]:
import pandas as pd
train_df = pd.read_csv('/Users/adam/Data_Analytics/Python/Datasets/naver_ratings/ratings_train.txt', sep='\t')
train_df.head(3)

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0


In [21]:
# 학습 데이터 세트의 0과 1의 Label 값 비율을 살펴보자. 1이 긍정, 0이 부정 감성이다.

train_df['label'].value_counts()

0    75173
1    74827
Name: label, dtype: int64

In [22]:
# 0과 1의 비율이 어느 한쪽으로 치우치지 않고 균등한 분포를 나타내고 있다.
# train_df의 경우 리뷰 텍스트를 가지는 'document' 컬럼에 Null이 존재하므로 이 값은 공백으로 변환한다.
# 문자가 아닌 숫자의 경우 단어적인 의미로 부족하므로 파이썬의 정규 표현식 모듀인 re를 이용해 이 역시 공백으로 변환한다.
# 테스트 데이터 세트의 경우도 파일을 로딩하고 동일한 데이터 가공을 수행한다.

In [23]:
import re

train_df = train_df.fillna(' ')
# 정규 표현식을 이용하여 숫자를 공백으로 변경(정규 표현식으로 \d 는 숫자를 의미함.) 
train_df['document'] = train_df['document'].apply( lambda x : re.sub("\d+", " ", x) )

# 테스트 데이터 셋을 로딩하고 동일하게 Null 및 숫자를 공백으로 변환
test_df = pd.read_csv('/Users/adam/Data_Analytics/Python/Datasets/naver_ratings/ratings_test.txt', sep='\t')
test_df = test_df.fillna(' ')
test_df['document'] = test_df['document'].apply( lambda x : re.sub("\d+", " ", x) )

# id 칼럼 삭제 수행
train_df.drop('id', axis=1, inplace=True) 
test_df.drop('id', axis=1, inplace=True)

In [24]:
# 이제는 TF-IDF 방식으로 단어를 벡터화 할 텐데, 먼저 각 문장을 한글 형태소 분석을 통해 형태소 단어로 토큰화 하자.
# 한글 형태소 엔진은 SNS 분석에 적합한 Twitter 클래스를 이용하자.
# Twitter 객체의 morphs() 메서드를 이용하면 입력 인자로 들어온 문장을 형태소 단어 형태로 토큰화해 list 객체로 반환한다.

In [25]:
from konlpy.tag import Okt

Okt = Okt()
def tw_tokenizer(text):
    # 입력 인자로 들어온 텍스트를 형태소 단어로 토큰화해 리스트 형태로 반환
    tokens_ko = twitter.morphs(text)
    return tokens_ko

In [28]:
# 사이킷런의 TfidfVectorizer를 이용해 TF-IDF 피처 모델을 생성한다.
# ngram은 (1,2), min_df=3, max_df는 상위 90%로 제한한다.

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

# Okt 객체의 morphs( ) 객체를 이용한 tokenizerfmf tkdyd. ngram_range는 (1, 2)
tfidf_vect = TfidfVectorizer(tokenizer=tw_tokenizer, ngram_range=(1, 2), min_df=3, max_df=0.9)
tfidf_vect.fit(train_df['document'])
tfidf_matrix_train = tfidf_vect.transform(train_df['document'])

/Users/adam/anaconda3/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [31]:
# 로지스틱 회귀를 이용해 감성 분석 분류 수행.
lg_clf = LogisticRegression(random_state=0, solver='liblinear')

# Parameter C 최적화를 위해 GridSearchCV 를 이용.
params = { 'C': [1, 3.5, 4.5, 5.5, 10]}
grid_cv = GridSearchCV(lg_clf, param_grid=params, cv=3, scoring='accuracy', verbose=1)
grid_cv.fit(tfidf_matrix_train, train_df['label'])
print(grid_cv.best_params_, round(grid_cv.best_score_, 4))

Fitting 3 folds for each of 5 candidates, totalling 15 fits
{'C': 3.5} 0.8593


In [33]:
# 테스트 세트를 이용해 최종 감성 분석 예측을 수행하자.
# 테스트 세트를 이용해 예측할 때는 학습할 때 적용한 TfidfVectorizer를 그대로 사용해야 한다.
# 그래야만 학습 시 설정된 TfidfVectorizer의 피처 개수와 테스트 데이터를 TfidfVectorizer로 변환할 피처 갯수가 같아진다.

In [34]:
from sklearn.metrics import accuracy_score

# 학습 데이터를 적용한 TfdifVectorizer를 이용하여 테스트 데이터를 TF-IDF 값으로 Feature 변환함.
tfidf_matrix_test = tfidf_vect.transform(test_df['document'])

# classifier는 GridSearchCV에서 최적 파라미터로 학습된 clssifier를 그대로 이용
best_estimator = grid_cv.best_estimator_
preds = best_estimator.predict(tfidf_matrix_test)

print('Logistic Regression 정확도:', accuracy_score(test_df['label'], preds))

Logistic Regression 정확도: 0.86172
